# PS 2


In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [ ]:
#ref_img is the image of object we need to find
ref_img = cv2.imread('imgr.PNG')
#converting to HSV from RGB
hsvr = cv2.cvtColor(ref_img, cv2.COLOR_BGR2HSV)

#finding histogram of ref_img using calcHist and normalizing
ref_hist = cv2.calcHist([hsvr], [0, 1], None, [180, 256], [0, 180, 0, 256])
cv2.normalize(ref_hist,ref_hist,0,255,cv2.NORM_MINMAX)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(ref_hist);

In [ ]:
def match_barrel(img):
    img1= cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    #apply backprojection
    res = cv2.calcBackProject([img1],[0,1],ref_hist,[0,180,0,256],1)
    
    #apply a convolution with a circular disc
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    cv2.filter2D(res,-1,disc,res)
    cv2.normalize(res,res,0,255,cv2.NORM_MINMAX)
    
    # Use thresholding to segment out the region
    ret,thresh = cv2.threshold(res,50,255,0)
    
    # Overlay images using bitwise_or
    thresh = cv2.merge((thresh,thresh,thresh))
    res1 = cv2.bitwise_or(img,thresh)
    return res1




In [ ]:
cap = cv2.VideoCapture('video.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    result = match_barrel(frame)
    cv2.imshow('a1', result)
    
    if cv2.waitKey(1) & 0xFF== ord('q'):
        break
cap.release()
cv2.destroyAllWindows()